In [ ]:
#facetrain.py    #################### File to train The Binary Pattern Recognizer ############################
import os
import PIL
from PIL import Image
import numpy as np
import cv2
import pickle



current_id = 0
label_ids ={}


#base_dir = os.path.dirname(os.path.abspath(__file__))
image_dir = os.path.join("images")
face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_alt2.xml")

recognizer = cv2.face.LBPHFaceRecognizer_create()


y_labels = []
x_train = []

for root,dirs,files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root,file)
            label = os.path.basename(os.path.dirname(path)).replace(" ","-")
            if not label in label_ids:
                
                label_ids[label] = current_id
                current_id = current_id+1
            
            id_ = label_ids[label]
            print(label_ids)
            
            pil_image = Image.open(path).convert("L")
            ##resizing kor
            size = (550,550)
            final_image = pil_image.resize(size,Image.ANTIALIAS)
            
            image_array = np.array(final_image,"uint8")
            #print(image_array)
            #print(path)
            #print(label,path)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            for(x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
    
#print(y_labels)
#print(x_train)

with open("labelslst.pickle",'wb') as f:
    pickle.dump(label_ids,f)
    
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainerlst.yml")



In [ ]:

import cv2
import pickle
import numpy as np

import urllib.request
import time


face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_alt2.xml")

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainerlst.yml")

labels = {"persons name":1}
with open("labelslst.pickle",'rb')as f:
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}


cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x,y,w,h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color=frame[y:y+h, x:x+w]
        
        id_ , conf = recognizer.predict(roi_gray)
        if conf>=85 and conf<95: #and conf<80:
            print(id_)
            print(labels[id_])
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            #############
            
            #link='http://helpmind.online/iot/get_from_py.php?h='+str(labels[id_])
            #urllib.request.urlopen(link)

            
            #############
            
            color = (255,255,255)
            stroke = 2
            cv2.putText(frame, name,(x,y),font,1,color,stroke,cv2.LINE_AA)
            #print(conf)
        else:
            print(id_)
            print(labels[id_])
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = "Unknown"#labels[id_]
            color = (255,255,255)
            stroke = 2
            cv2.putText(frame, name,(x,y),font,1,color,stroke,cv2.LINE_AA)
            
            
        
        
        

        #img_item = "my-image.png"
        #cv2.imwrite(img_item,roi_gray)
        color = (255,0,0)
        stroke = 4
        end_cord_x= x+w
        end_cord_y = y+h
        
        cv2.rectangle(frame,(x,y),(end_cord_x,end_cord_y),color,stroke)
        
    cv2.imshow('frame',frame)
        #cv2.waitKey()
        #cv2.destroyAllWindows()
    if cv2.waitKey(20)& 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

#x = datetime.datetime.now()
#print(x)

#link='http://helpmind.online/iot/get_from_py.php?h='+str(x)
#urllib.request.urlopen(link)
